Prepare the data for human lisbet experiences

In [1]:
import pandas as pd
import os
import h5py
import math
import random
import numpy as np
import json

SEED = 42

In [2]:
root = r"./datasets/humans"


In [3]:
filename = os.path.join(root, "humans_TD.h5")
a=h5py.File(filename)

d_TD = []
for key, vh in a.items():
    data = a[key]['keypoints'][()]
    point = (key, {'keypoints':data})
    d_TD.append(point)
a.close()
print(len(d_TD))

30


In [4]:
filename = os.path.join(root, "humans_ASD.h5")
a=h5py.File(filename)

d_ASD = []
for key, vh in a.items():
    data = a[key]['keypoints'][()]
    point = (key, {'keypoints':data})
    d_ASD.append(point)
a.close()
print(len(d_ASD))

60


In [5]:
# V1 all
d5 = d_TD + d_ASD

filename = os.path.join(root, "humans_all.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in d5:
        fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])

In [6]:
filename = os.path.join(root, "humans_all.h5")

with h5py.File(filename, 'r') as fd:

    d_5 = []
    for key, vh in fd.items():
        data = fd[key]['keypoints'][()]
        point = (key, {'keypoints':data})
        d_5.append(point)

print(len(d_5))

90


In [7]:
# V1 50/50
size = min(len(d_TD), len(d_ASD))
random.seed(SEED)
random.shuffle(d_TD)
random.shuffle(d_ASD)
d5 = d_TD[:size] + d_ASD[:size]

filename = os.path.join(root, "humans_50-50.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in d5:
        fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])

In [8]:
filename = os.path.join(root, "humans_50-50.h5")

with h5py.File(filename, 'r') as fd:

    d_5 = []
    for key, vh in fd.items():
        data = fd[key]['keypoints'][()]
        point = (key, {'keypoints':data})
        d_5.append(point)

print(len(d_5))

60


In [14]:
new_d_5 = []
dm = pd.read_excel(os.path.join(root, r"data_mapping.xlsx"), dtype=str)
dm = dm[dm['From']=='Sara'] 

dm = dm.drop_duplicates()


dm['video'] = dm['VCFS_DATABASE_ADMIN 2::Sujet_ID']
dm['diagnosis'] = dm['VCFS_DATABASE_ADMIN 2::Diagnosis']
dm['diagnosis'] = dm['diagnosis'].replace({'Low-Risk':'TD'})
dm['diagnosis'] = dm['diagnosis'].replace({'Normal_Control':'TD'})
dm['diagnosis'] = dm['diagnosis'].replace({'Autism':'ASD'})

dm['diagnosis'] = dm['diagnosis'].astype('category')

dm['diag_int'] = dm['diagnosis'].cat.codes
category_mapping = dict(enumerate(dm['diagnosis'].cat.categories))

for id, vid_dic in d_5:
    vid_dic['diag'] = dm[dm['video']==id]['diag_int'].to_numpy()
    new_d_5.append((id, vid_dic))

In [20]:
filename = os.path.join(root, "humans_50-50_annoted.h5")

with h5py.File(filename, 'w') as fd:
    for key, val in new_d_5:
        ds = fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])
        ds.attrs.create('diag', data=val["diag"])


filename = os.path.join(root, "humans_50-50_annoted.mapping.json")
with open(filename, 'w') as fd:
    json.dump(category_mapping, fd, indent=4)

filename = os.path.join(root, "humans_50-50_annoted.label.json")
with open(filename, 'w') as fd:
    json.dump(dm[['diagnosis', 'video']].to_dict(), fd, indent=4)

In [19]:
dm[['diagnosis', 'video']].to_dict()

{'diagnosis': {0: 'TD',
  2: 'TD',
  3: 'TD',
  4: 'ASD',
  5: 'ASD',
  6: 'TD',
  8: 'TD',
  9: 'TD',
  10: 'TD',
  11: 'TD',
  12: 'ASD',
  13: 'TD',
  14: 'ASD',
  15: 'ASD',
  16: 'TD',
  17: 'ASD',
  18: 'ASD',
  19: 'ASD',
  22: 'TD',
  23: 'TD',
  24: 'TD',
  25: 'ASD',
  27: 'ASD',
  30: 'ASD',
  31: 'ASD',
  32: 'ASD',
  33: 'ASD',
  37: 'ASD',
  38: 'ASD',
  39: 'ASD',
  41: 'ASD',
  42: 'ASD',
  43: 'ASD',
  45: 'ASD',
  47: 'ASD',
  48: 'TD',
  49: 'ASD',
  50: 'ASD',
  51: 'TD',
  52: 'TD',
  53: 'ASD',
  54: 'TD',
  56: 'TD',
  58: 'TD',
  59: 'TD',
  60: 'ASD',
  61: 'TD',
  62: 'TD',
  63: 'ASD',
  64: 'ASD',
  66: 'ASD',
  67: 'ASD',
  68: 'ASD',
  69: 'TD',
  70: 'ASD',
  72: 'ASD',
  73: 'ASD',
  74: 'ASD',
  76: 'TD',
  77: 'TD',
  78: 'ASD',
  79: 'ASD',
  80: 'ASD',
  81: 'ASD',
  82: 'ASD',
  83: 'ASD',
  84: 'ASD',
  85: 'ASD',
  86: 'ASD',
  87: 'ASD',
  88: 'ASD',
  89: 'ASD',
  90: 'ASD',
  91: 'ASD',
  92: 'ASD',
  93: 'ASD',
  94: 'ASD',
  95: 'ASD',
  96: 

In [5]:
# to try and shuffle id

# d_n = []

# proba_shuffle = 1
# seed = 42
# random.seed(seed)
# for frame_id, dict_keyp in d_5:
#     keypoints = dict_keyp['keypoints']
#     new_kp = keypoints.copy()
#     for i in range(len(keypoints)):
#         if random.random() < proba_shuffle:
#             new_kp[i] = np.concatenate((keypoints[i][17:], keypoints[i][:17]))

#     d_n.append((frame_id, {'keypoints':new_kp}))

# filename = os.path.join(root, "humans_50-50_skeletonIDshuffle_p100.h5")

# with h5py.File(filename, 'w') as fd:
#     for key, val in d_n:
#         fd.create_dataset(f"{key}/keypoints", data=val["keypoints"])

In [63]:
# filename = os.path.join(root, "humans_50-50_skeletonIDshuffle_p50.h5")

# with h5py.File(filename, 'r') as fd:

#     d_5 = []
#     for key, vh in fd.items():
#         data = fd[key]['keypoints'][()]
#         point = (key, {'keypoints':data})
#         d_5.append(point)

# print(len(d_5))

60
